## Bloc consacré au cleaning du text d'un tweet

In [1]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import tokenize
import numpy as np

punctuations=string.punctuation.replace('@','')
nltk.download('stopwords')
stop = stopwords.words('english')
stop.append('rt')
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mamin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mamin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
def cleaning(text):
    text=text.lower()
    #Enlever la ponctuation
    text=re.sub('[%s]' % re.escape(punctuations), '',text)
    #Enlever les mots vides + les mentions "@"
    text=" ".join([word for word in str(text).split() if not (word in stop or word[0]=="@" or word[:4]=="http")])
    #Word stemming
    text=" ".join([stemmer.stem(word) for word in text.split()])
    #Lemmatization
    text=" ".join([lemmatizer.lemmatize(word) for word in text.split()])
    return text

### On récupère la BD des villes et on s'authetifie sur l'API twitter

In [3]:
import tweepy
import pandas as pd

In [19]:
df_country = pd.read_csv('Villes/Villes.csv')
df_country.head()

,Unnamed: 0,name,state_name,state_code,country_id,country_code,country_name
0,138883,New York City,New York,NY,233,USA,United States
1,129652,"Washington, D.C.",District of Columbia,DC,233,USA,United States
2,128848,San Francisco,California,CA,233,USA,United States
3,37070,Berlin Köpenick,Berlin,BE,82,DE,Germany
4,37069,Berlin,Berlin,BE,82,DE,Germany


In [5]:
# Les identifiants de l'API twitter que l'on récupere sur Twitter Developper Platform

api_key = "gMJX3zW3HN3nOgQChObsQiylc"
api_secret = "u39QEZJV390xVZBGtldHPjNnkhwD0G5EQnJP4FSbN0RiLeMSQo"
access_token = "1255191890653327360-YCXKJgGWUZ7AnWmFwMdqVZQkwGnd4w"
access_token_secret = "KGHPuaWcOfiKJvLmNXZgL2VsObD8vox7R3APBmS45IsxU"


# Authetification
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

### Procédure de récuperation du pays du publicateur du tweet

In [6]:
import re
def split_loc(loc):
    L=re.split(',+',loc)
    return [e.strip() for e in L]

In [57]:
"""
J'ai remarqué que la structure générale de localisation est Ville, Nom of State, State Code, Pays.
Chacun de ses éléments pouvant exister ou non:
-Paris, France
-Japan
-Manhattan, NY
Ou même quelque chose n'ayant aucun rapport: "The streets" par exemple.

"""
def get_Country(x):
    if len(x)>0:
        #Via pays
        z=x[-1]
        L=list(df_country[df_country['country_name']==z]['country_name'].unique())
        if len(L)==1:
            return L[0]
        
        else:
            #Via Country Code
            if z=="US":
                z="USA"
            L=list(df_country[df_country['country_code']==z]['country_name'].unique())
            if len(L)==1:
                return L[0]
            
            else:
                #Via State Code
                if z=="NYC":
                    z="NY"
                M=list(df_country[df_country['state_code']==z]['country_name'].unique())
                if len(M)==1:
                    return M[0]

                else:
                    #Via State Name
                    M=list(df_country[df_country['state_name']==z]['country_name'].unique())
                    if len(M)==1:
                        return M[0]

                    else:
                        #Via Ville
                        z=x[0]
                        M=list(df_country[df_country['name']==z]['country_name'].unique())
                        if len(M)==1 :
                            return M[0]
                        else:
                            return np.nan

In [8]:
keyword = input("Veuillez entrer le mot clé sur lequel se basera l'étude: " )+" lang:en"

Veuillez entrer le mot clée sur lequel se basera l'étude: bitcoin


In [21]:
tweet_list=[]

### Création de la base de donnée des Tweets    

In [22]:
"""
Twitter applique une restriction, de ne pas récupérer un certain nombre de tweets, on demande donc au programme en cas d'erreur de patienter
pendant 15 minutes avant de réeffectuer des recherches.
"""
from time import sleep
from afinn import Afinn
afn = Afinn()
limit=178
l=0
k=0
for i in range(50):
    try:
        tweets = tweepy.Cursor(api.search_tweets, q=keyword).items(limit)
        for tweet in tweets:
            id=tweet.user.id_str
            country=get_Country(split_loc(tweet.user.location))
            text=cleaning(tweet.text)
            sentiment_value=afn.score(text)
            if sentiment_value > 0:
                sentiment="Positif"

            elif sentiment_value < 0:
                sentiment="Negatif"
            else:
                sentiment="Neutral"
            tweet_list.append([id,country,text,sentiment])
        l+=1
        print("Execution numero :",l)
        sleep(30)
    except:
        k+=1
        print("Sleep numero :",k)
        sleep(900)

Execution numero : 1
Execution numero : 2


In [23]:
id_list=[e[0] for e in tweet_list]
country_list=[e[1] for e in tweet_list]
text_list=[e[2] for e in tweet_list]
sentiment_list=[e[3] for e in tweet_list]

In [24]:
dict={'ID':id_list,
      'Country':country_list,
      'Text':text_list,
      'Sentiment':sentiment_list}
df=pd.DataFrame(dict)

In [26]:
df=df.dropna().drop_duplicates()

In [54]:
#Liste des pays non disponibles dans le module pygal sur lequel on affichera une carte identifiant l'avis global du pays
L=["South Sudan","Nauru","Tuvalu","Saint Kitts And Nevis","Bonaire","Sint Eustatius and Saba","Palau","Kiribati","Virgin Islands (US)","Micronesia","Grenada","Saint Vincent And The Grenadines","Solomon Islands","Trinidad And Tobago","Tonga","Vanuatu","Dominica","Antigua And Barbuda","Samoa","Barbados","Fiji Islands","Saint Lucia","East Timor","Comoros","Qatar","Nicaragua","The Bahamas"]

In [56]:
df.drop(df[df.Country.isin(L)].index,inplace=True)

In [44]:
fdf.to_csv('Tweets.csv')